In [2]:
import numpy as np
import networkx as nx
import scipy.linalg as la
import nb_general as nb
import matplotlib.pyplot as plt
import non_backtracking_tools as nbt

In [3]:
# Things to check
xy = []
lower = []
upper = []
lower_k = []
avg_k= []
upper_k = []
diagonal = []

In [29]:
# Create 1000 random graphs
for _ in range(1000):
    G = nx.random_partition_graph([50,50],0.25,.05)    # Create adjacency matrix
    A = nx.adjacency_matrix(G).todense()
    D = np.diag(list(dict(G.degree).values()))
    D_inv = np.diag([1/i for i in dict(G.degree).values()])
    # Create K
    K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])
    # Find nonbacktracking mixing rate
#     P = nbt.trans_prob_matrix_nb(G)
    # Get all necessary eigenvalues and eigenvectors
    lambda_vals, lambda_vecs = la.eig(A)
    theta_vals, theta_vecs = la.eig(D_inv@A)
    mu_vals, mu_vecs = la.eig(K)
#     rho_vals, rho_vecs = la.eig(P)
    # Define x and y
    x = lambda_vecs[:,1]
    y = mu_vecs[mu_vecs.shape[0]//2:,1]
    # Get max, min, and average degree
    d_max = max(list(dict(G.degree).values()))
    d_min = min(list(dict(G.degree).values()))
    d_bar = np.mean(list(dict(G.degree).values()))
    
    lambda_2 = np.max(np.abs(lambda_vals[1:]))
    mu_2 = np.max(np.abs(mu_vals[1:]))
    theta_2 = np.max(np.abs(theta_vals[1:]))
#     rho_2 = np.max(np.abs(rho_vals[1:]))
    
    # Check various ideas
    
    # Check that x^Ty\neq 0
    if x.T@y == 0:
        xy.append(G)
    
    # Check that \lambda_2/d_min \geq \theta_2
    if lambda_2/d_min < theta_2:
        upper.append(G)
    
    # Checkt that \lambda_2/d_max \leq \theta_2
    if lambda_2/d_max > theta_2:
        lower.append(G)
    
    # Check upper bound of k
    # Make sure that this is the actually teh second largest
    # eigenvalue. Also compare the absolute values of both sides
    # to deal with complex numbers. In other words, compare
    if mu_2.dtype != float:
        if np.abs(mu_2) > np.abs((lambda_2 + np.sqrt(lambda_2**2-4*(d_min-1)))/2):
            upper_k.append(G)
    
    # Check lower bound of k
    if np.abs(mu_2) < np.abs((lambda_2 + np.sqrt(lambda_2**2-4*(d_max-1)))/2):
        lower_k.append(G)
        
#     U, s, V = la.svd(K)
#     if len(s) != K.shape[1]:
#         diagonal.append(G)
    

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in sqrt


In [ ]:
len(xy)

In [ ]:
len(upper)

In [ ]:
len(lower)

In [ ]:
len(upper_k)

In [ ]:
len(lower_k)

In [ ]:
len(diagonal)

In [ ]:
d_bar

In [5]:
X = np.array([[1,1],[0,1]])
la.eig(la.eig(X)[1])

(array([  1.00000000e+00+0.j,   2.22044605e-16+0.j]),
 array([[ 1.        ,  0.70710678],
        [ 0.        ,  0.70710678]]))

In [13]:
break_mu = [la.eig(np.block([[nx.adjacency_matrix(i).todense(),np.diag(list(dict(G.degree).values()))-np.eye(len(i.degree))],[-np.eye(len(i.degree)),np.zeros((len(i.degree),len(i.degree)))]]))[0][1] for i in upper_k]

In [15]:
for i in break_mu:
    print(i)

(-4.44537199537+2.85340902592j)
(-4.50866306352+2.7120848437j)
(-4.66600027843+2.39123604401j)
(4.20905975479+3.10977674251j)
(-4.49328109459+2.66881272628j)
(4.23714561632+3.0700524714j)
(4.23851510591+2.94881676585j)
(-4.38021418965+2.77960284784j)
(4.29716279329+3.04780652349j)
(4.27622047138+2.95022708197j)
(4.15968985242+2.99565592242j)
(4.03322060433+3.27654563447j)
(4.10648849071+3.12935611188j)
(4.22008082639+3.0936194019j)
(4.07408516314+3.44143093842j)
(-4.48257233047+2.77077741926j)
(3.8788394428+3.44658837408j)
(4.26494579875+2.96866236348j)
(-4.55609225019+2.51683792053j)
(-4.52772989058+2.746909221j)
(-4.50579710943+2.60763869381j)
(4.30911141741+2.83348733268j)
(-4.56216530638+2.26906568076j)
(4.06443288783+3.12751394997j)
(4.18212212154+3.04045086968j)
(4.10866428908+3.15286031118j)
(4.13351048815+3.20171458058j)
(-4.49568998573+2.41792506237j)
(-4.50952211441+2.60834462736j)
(-4.53809832557+2.49638344201j)
(-4.55440692848+2.47818220664j)
(-4.50248680496+2.57697210655j)

In [46]:
break_mu[0] < break_mu[1]

True

In [23]:
np.abs(break_mu[0])

5.2823550852321048

In [9]:
# Find nonbacktracking mixing rate
P = nbt.trans_prob_matrix_nb(G)

In [10]:
# Get eigenvalues
rho_vals, rho_vecs = la.eig(P)

In [13]:
rho_2 = rho_vals[1]

In [15]:
lambda_2 = la.eig(A)[0][1]

In [16]:
d_bar

15.279999999999999

In [18]:
# Check bound
x = la.eig(A)[1][:,1]

In [25]:
c = x.T@y

In [26]:
d = x.T@(D-np.eye(D.shape[0]))@y

In [27]:
(lambda_2*c+np.sqrt((lambda_2*c)**2-4*d_bar))/(2*(d_bar))

(-0.031128738558430659-0.25392130240247929j)

In [28]:
rho_2

(0.60946117975858494+0j)

# Bound on $\rho_2$ does not hold